# Amazon Braket Spending Limits: An Introduction

## Introduction

Amazon Braket's spending limits functionality provides essential budget control for quantum computing experiments. This feature helps prevent unexpected charges by allowing you to set maximum spending thresholds for specific quantum devices.

### How Spending Limits Work
- **Device-Specific**: Each quantum device can have its own spending limit
- **Cumulative Tracking**: Limits track total spending across all tasks
- **Execution Timing**: A time-window for execution can be prescribed
- **Pre-Execution Check**: Limits are checked before task submission
- **Real-Time Enforcement**: Tasks are rejected if they would exceed the limit or fall outside of the prescribed time-window

### Why Use Spending Limits
- **Budget Management**: Control costs during experimentation and development
- **Educational Use**: Prevent accidental overspending in learning environments
- **Production Safety**: Add guardrails for automated quantum workflows
- **Team Collaboration**: Ensure shared resources stay within allocated budgets

### Available Management Methods
Currently, the Amazon Braket SDK does not provide wrappers for interacting with the spending limits. As such, there are three ways to access Braket to configure spending limits:
1. **AWS Console** - Web-based graphical interface
2. **AWS CLI** - Command-line interface for configuration
3. **AWS SDK (boto3)** - Programmatic access from Python applications

### ⚠️ Important Notice
This notebook will create or temporarily modify spending limits for demonstration purposes. This may interfere with other quantum computing tasks running from the same AWS account. Please coordinate with your team if you share an AWS account for quantum computing workloads.

In [1]:
# Import required libraries
from braket.aws import AwsDevice
from braket.aws.queue_information import QueueType
from braket.circuits import Circuit
from braket.devices import Devices
import json
import pprint

# Configuration constants
DEMO_SPENDING_LIMIT = 3.0  # Demo budget in USD
HIGH_COST_SHOTS = 10000    # Shot count that exceeds our budget
LOW_COST_SHOTS = 500       # Shot count within our budget
MAX_QUEUE_THRESHOLD = 10   # Maximum acceptable queue depth

**NOTE:** This notebook uses boto3 for programmatic API access. Equivalent AWS CLI commands are provided for reference throughout the demonstration.

In [2]:
# Import boto3 for programmatic API access
import boto3
print("✅ boto3 is available - using programmatic API access")

✅ boto3 is available - using programmatic API access


## Choosing your QPU Device
Amazon Braket allows users to set different spending limits for different quantum devices supported by the service. 

We'll use Rigetti's Ankaa-3 device for this demonstration. This device offers:
- Reliable availability for testing
- Clear per-shot pricing model
- Good performance for educational examples

To select a different device - change the ARN in the cell below.

In [3]:
device_arn = Devices.Rigetti.Ankaa3

In [4]:
# Select and initialize the quantum device
region = AwsDevice.get_device_region(device_arn)
device = AwsDevice(device_arn)

print(f"🔧 Selected Device: {device_arn}")
print(f"🌍 Region: {region}")
print(f"📊 Device Status: {'Available' if device.is_available else 'Unavailable'}")

# Display current queue information
try:
    queue_depth = device.queue_depth()
    print(f"📋 Queue Status: {queue_depth.jobs} total jobs")
    print(f"   - Normal priority: {queue_depth.quantum_tasks[QueueType.NORMAL]}")
    print(f"   - High priority: {queue_depth.quantum_tasks[QueueType.PRIORITY]}")
except Exception as e:
    print(f"⚠️  Could not retrieve queue information: {e}")

🔧 Selected Device: arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3
🌍 Region: us-west-1
📊 Device Status: Available
📋 Queue Status: 0 total jobs
   - Normal priority: 1
   - High priority: 0


## Managing spending limits

### Searching for existing spending limits

First let us see if there is already a spending limit set for the device.

In [6]:
# Configure search filters for the specific device
search_filters = [
    {
        'name': 'deviceArn',
        'values': [device_arn],
        'operator': 'EQUAL'
    }
]

# Initialize Braket client
braket_client = boto3.client("braket", region_name=region)

print("🔍 Searching for existing spending limits...\n")
print("📋 Equivalent AWS CLI command:")
print(f"aws --region {region} braket search-spending-limits --filters='{json.dumps(search_filters)}'\n")

# Execute the search
try:
    response = braket_client.search_spending_limits(filters=search_filters)
    spending_limits = response['spendingLimits']
except Exception as e:
    print(f"❌ Error searching spending limits: {e}")
    spending_limits = []

print(f"✅ Search completed - found {len(spending_limits)} existing limit(s)")

🔍 Searching for existing spending limits...

📋 Equivalent AWS CLI command:
aws --region us-west-1 braket search-spending-limits --filters='[{"name": "deviceArn", "values": ["arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3"], "operator": "EQUAL"}]'

✅ Search completed - found 1 existing limit(s)


In [8]:
# Process search results
existing_limit_amount = None
existing_spending_limit = None

if len(spending_limits) > 0:
    # Should only be one limit per device
    assert len(spending_limits) == 1, f"Expected 1 limit, found {len(spending_limits)}"
    
    existing_spending_limit = spending_limits[0]
    existing_limit_amount = existing_spending_limit['spendingLimit']
    
    print(f"💰 Existing spending limit: ${existing_limit_amount}")
    print("📊 Current usage:")
    print(f"   - Total spent: ${existing_spending_limit.get('totalSpend', '0.00')}")
    print(f"   - Queued spend: ${existing_spending_limit.get('queuedSpend', '0.00')}")
    print("\n📋 Full details:")
    pprint.pprint(existing_spending_limit)
else:
    print("🆕 No existing spending limit found for this device")
    print("   We'll create a new one in the next step")

💰 Existing spending limit: $10
📊 Current usage:
   - Total spent: $0
   - Queued spend: $0

📋 Full details:
{'createdAt': datetime.datetime(2025, 11, 3, 19, 12, 46, 135000, tzinfo=tzlocal()),
 'deviceArn': 'arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3',
 'queuedSpend': '0',
 'spendingLimit': '10',
 'spendingLimitArn': 'arn:aws:braket:us-west-1:656977514066:spending-limit/01e218e4-7393-4172-a76e-8f9060594009',
 'tags': {},
 'timePeriod': {'endAt': datetime.datetime(2125, 12, 30, 0, 0, tzinfo=tzlocal()),
                'startAt': datetime.datetime(2025, 11, 3, 19, 12, 46, tzinfo=tzlocal())},
 'totalSpend': '0',
 'updatedAt': datetime.datetime(2025, 11, 4, 16, 11, 57, 977000, tzinfo=tzlocal())}


## Creating / Updating Spending Limits

Now we will allocate a \$3 budget for ourselves. We do that by either updating the existing spending limit or by creating a new one if no existing spending limit was found.

In [9]:
def calculate_safe_limit(existing_limit):
    """
    Calculate a safe spending limit that accounts for existing usage.
    
    Args:
        existing_limit: Dictionary containing existing limit information
        
    Returns:
        float: Safe limit amount that won't immediately conflict
    """
    if not existing_limit:
        return DEMO_SPENDING_LIMIT
    
    total_spent = float(existing_limit.get('totalSpend', 0))
    queued_spend = float(existing_limit.get('queuedSpend', 0))
    current_usage = total_spent + queued_spend
    
    return current_usage + DEMO_SPENDING_LIMIT

# Determine the appropriate action and limit amount
new_limit_amount = calculate_safe_limit(existing_spending_limit)

if not existing_limit_amount:
    # Create new spending limit
    print(f"🆕 Creating new spending limit: ${new_limit_amount:.2f}\n")
    print("📋 Equivalent AWS CLI command:")
    print(f"aws --region {region} braket create-spending-limit "
          f"--device-arn {device_arn} "
          f"--spending-limit {new_limit_amount:.2f}\n")
    
    try:
        response = braket_client.create_spending_limit(
            deviceArn=device_arn, 
            spendingLimit=f"{new_limit_amount:.2f}"
        )
        spending_limit_arn = response['spendingLimitArn']
        print("✅ Successfully created spending limit")
    except Exception as e:
        print(f"❌ Error creating spending limit: {e}")
        spending_limit_arn = None
        
else:
    # Update existing spending limit
    spending_limit_arn = existing_spending_limit['spendingLimitArn']
    
    print(f"🔄 Updating existing spending limit from ${existing_limit_amount} "
           f"to ${new_limit_amount:.2f}\n")
    print("📋 Equivalent AWS CLI command:")
    print(f"aws --region {region} braket update-spending-limit "
          f"--spending-limit-arn {spending_limit_arn} "
          f"--spending-limit {new_limit_amount:.2f}\n")
    
    try:
        response = braket_client.update_spending_limit(
            spendingLimitArn=spending_limit_arn, 
            spendingLimit=f"{new_limit_amount:.2f}"
        )
        print("✅ Successfully updated spending limit")
    except Exception as e:
        print(f"❌ Error updating spending limit: {e}")

🔄 Updating existing spending limit from $10 to $3.00

📋 Equivalent AWS CLI command:
aws --region us-west-1 braket update-spending-limit --spending-limit-arn arn:aws:braket:us-west-1:656977514066:spending-limit/01e218e4-7393-4172-a76e-8f9060594009 --spending-limit 3.00

✅ Successfully updated spending limit


In [10]:
print(f"🔗 ARN: {spending_limit_arn}")

🔗 ARN: arn:aws:braket:us-west-1:656977514066:spending-limit/01e218e4-7393-4172-a76e-8f9060594009


## Testing Cost Controls

We will now attempt to submit quantum tasks, using a simple circuit (state-preparation for a uniform superposition of two qubits). We will control the cost of the tasks by changing the number of shots we wish to perform.

**NOTE**: We chose to modify the cost of task execution for the purpose of demonstrating the real-time spending limit enforcement. One could also experiment with modifying the time period of the spending limit (for example, we can set it to start one day in the future by updating the `["timePeriod"]["startAt"]` entry of the spending limit and setting it to `str(datetime.datetime.now() + datetime.timedelta(days=1))`).

In [11]:
# Create a simple test circuit
test_circuit = Circuit().h(0).h(1)

print("Test Circuit:")
print(test_circuit)

Test Circuit:
T  : │  0  │
      ┌───┐ 
q0 : ─┤ H ├─
      └───┘ 
      ┌───┐ 
q1 : ─┤ H ├─
      └───┘ 
T  : │  0  │


### Task outside of our budget

In [12]:
print("\n💰 Cost Analysis:")
print("   - Per-shot cost: $0.0009")
print("   - Per-task cost: $0.30")
print(f"   - Requested shots: {HIGH_COST_SHOTS:,}")
print(f"   - Estimated cost: ~${0.30 + HIGH_COST_SHOTS * 0.0009:.2f}")
print(f"   - Available budget: ${DEMO_SPENDING_LIMIT:.2f}")
print("   - Expected result: ❌ Task should be rejected")


💰 Cost Analysis:
   - Per-shot cost: $0.0009
   - Per-task cost: $0.30
   - Requested shots: 10,000
   - Estimated cost: ~$9.30
   - Available budget: $3.00
   - Expected result: ❌ Task should be rejected


In [13]:
# Attempt to submit the high-cost task
print(f"🚀 Attempting to submit task with {HIGH_COST_SHOTS:,} shots...\n")

try:
    job = device.run(test_circuit, shots=HIGH_COST_SHOTS)
    print(f"⚠️  Unexpected: Task was accepted! Job ID: {job.id}")
    print("   This might indicate the spending limit isn't active yet")
    
except Exception as e:
    print("✅ Expected behavior: Task was rejected.")
    print("\n📋 Exception details:")
    print(f"   Type: {type(e).__name__}")
    print(f"   Message: {str(e)}")
    
    # Check if it's specifically a spending limit error
    if "spending limit" in str(e).lower():
        print("\n🎯 Confirmed: This is a spending limit enforcement")
    else:
        print("\n❓ Note: This might be a different type of error")

🚀 Attempting to submit task with 10,000 shots...

✅ Expected behavior: Task was rejected.

📋 Exception details:
   Type: ValidationException
   Message: An error occurred (ValidationException) when calling the CreateQuantumTask operation: Attempted to spend over the spending limit or outside the spending limit time window. Spending limit: 3, current spend: 0, estimated cost: 9.3000000000, start time: 2025-11-03T19:12:46.130Z, end time: 2125-12-30T00:00:00.000Z

🎯 Confirmed: This is a spending limit enforcement


### Task within our budget

In [14]:
# Test with a lower shot count that should be within budget
low_cost_estimate = 0.30 + LOW_COST_SHOTS * 0.0009

print("🧪 Testing with acceptable cost:")
print(f"   - Shots: {LOW_COST_SHOTS:,}")
print(f"   - Estimated cost: ~${low_cost_estimate:.2f}")
print("   - Expected result: ✅ Should be accepted\n")

🧪 Testing with acceptable cost:
   - Shots: 500
   - Estimated cost: ~$0.75
   - Expected result: ✅ Should be accepted



⚠️ *Note*: Executing the following code will submit a task incurring a cost. To make sure this is not done unintentionally the cell below is in markdown by default.

```python
# Check device availability first
if device.is_available:
    queue_depth = device.queue_depth()
    total_queue = (int(queue_depth.quantum_tasks[QueueType.NORMAL]) + 
                   int(queue_depth.quantum_tasks[QueueType.PRIORITY]))
    
    if total_queue < MAX_QUEUE_THRESHOLD:
        print(f"📊 Device status: Available (queue: {total_queue} tasks)")
        print("🚀 Submitting low-cost task...\n")
        
        try:
            job = device.run(test_circuit, shots=LOW_COST_SHOTS)
            print(f"✅ Success! Task accepted with Job ID: {job.id}")
            print(f"📊 Task status: {job.state()}")
            print("\n💡 This confirms spending limits allow reasonable usage")
            
        except Exception as e:
            print(f"❌ Unexpected: Low-cost task was also rejected")
            print(f"   Error: {e}")
            print("   This might indicate a configuration issue")
    else:
        print(f"⏳ Device queue is busy ({total_queue} tasks) - skipping submission")
        print("   You can try again when the queue is less busy")
else:
    print("🔒 Device is currently unavailable - cannot test task submission")
    print("   This is normal and doesn't affect spending limit functionality")
```

## Cleanup

It's important to clean up the changes made during this demonstration. We'll either:
- **Delete** the spending limit if we created a new one
- **Restore** the original limit if we updated an existing one

In [15]:
# Cleanup: Restore original state
print("🧹 Cleaning up demonstration changes...\n")

if not existing_limit_amount:
    # Delete the spending limit we created
    print("🗑️  Deleting the spending limit we created\n")
    print("📋 Equivalent AWS CLI command:")
    print(f"aws --region {region} braket delete-spending-limit "
          f"--spending-limit-arn {spending_limit_arn}\n")
    
    try:
        response = braket_client.delete_spending_limit(
            spendingLimitArn=spending_limit_arn
        )
        print("✅ Successfully deleted spending limit")
        print("🔄 Device spending is now unrestricted")
    except Exception as e:
        print(f"❌ Error deleting spending limit: {e}")
        print("⚠️  You may need to delete it manually")
        
else:
    # Restore the original spending limit
    print(f"🔄 Restoring original spending limit: ${existing_limit_amount}\n")
    print("📋 Equivalent AWS CLI command:")
    print(f"aws --region {region} braket update-spending-limit "
          f"--spending-limit-arn {spending_limit_arn} "
          f"--spending-limit {existing_limit_amount}\n")
    
    try:
        response = braket_client.update_spending_limit(
            spendingLimitArn=spending_limit_arn, 
            spendingLimit=existing_limit_amount
        )
        print(f"✅ Successfully restored original limit: ${existing_limit_amount}")
    except Exception as e:
        print(f"❌ Error restoring spending limit: {e}")
        print("⚠️  You may need to restore it manually")

print("\n🎉 Cleanup completed!")

🧹 Cleaning up demonstration changes...

🔄 Restoring original spending limit: $10

📋 Equivalent AWS CLI command:
aws --region us-west-1 braket update-spending-limit --spending-limit-arn arn:aws:braket:us-west-1:656977514066:spending-limit/01e218e4-7393-4172-a76e-8f9060594009 --spending-limit 10

✅ Successfully restored original limit: $10

🎉 Cleanup completed!


## Next Steps

Now that you understand the basics of spending limits, consider exploring:

- [Amazon Braket Pricing](https://aws.amazon.com/braket/pricing/)
- [Amazon Braket Developer Guide](https://docs.aws.amazon.com/braket/)
- [Amazon Braket Examples](https://github.com/aws/amazon-braket-examples)
- [AWS Cost Management](https://aws.amazon.com/aws-cost-management)

---

**📝 Note**: This notebook demonstrated spending limit management in a safe, controlled manner. Always review costs and limits before running production quantum workloads.